# 数据增强之深入理解 RunnableParallel

### 版权声明：本文为CSDN博主「AI架构师易筋」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
### 原文链接：https://blog.csdn.net/zgpeace/article/details/135290412

In [2]:
import time

def measure_execution_time(func, *arg, **kwargs):
    """测量并返回给定函数的执行时间。

    Args:
        func: 要测量的函数对象。
        *args: 传递给函数的位置参数。
        **kwargs: 传递给函数的关键字参数。

    Returns:
        tuple: (执行结果, 耗时秒数)
    """
    
    start_time = time.perf_counter()
    process = func(*arg, **kwargs)
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    return process,elapsed_time


In [8]:
def sheep_counting(sheep_number):
    result = 0
    for i in range(sheep_number+1):
        result = result + i

    return result

In [9]:
result, pass_time = measure_execution_time(sheep_counting,10)
print(result,pass_time)

55 4.999979864805937e-06


In [4]:
import os

from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(
    openai_api_key=api_key,
    temperature=0.8,
    base_url="https://apejhvxcd.cloud.sealos.io/v1"
)

chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model 
chain2 = ChatPromptTemplate.from_template("write a poem in two lines about  {topic}") | model

combined = RunnableParallel(joke=chain1, poem=chain2) 

result, pass_time = measure_execution_time(combined.invoke,{"topic":"bears"})
print(f"执行结果: {result}")
print(f"函数执行耗时: {pass_time} 秒")

执行结果: {'joke': AIMessage(content="Why don't bears wear socks?\n\nBecause they have bear feet!"), 'poem': AIMessage(content='Majestic in woods, their grace does astound,\nBears, fierce and gentle, where wild hearts are found.')}
函数执行耗时: 1.5167087999871 秒


In [5]:
result, pass_time = measure_execution_time(chain1.invoke,{"topic":"bears"})
print(f"执行结果: {result}")
print(f"函数执行耗时: {pass_time} 秒")

执行结果: content="Why don't bears wear shoes?\n\nBecause they have bear feet!"
函数执行耗时: 1.1170307000284083 秒


In [6]:
result, pass_time = measure_execution_time(chain2.invoke,{"topic":"bears"})
print(f"执行结果: {result}")
print(f"函数执行耗时: {pass_time} 秒")

执行结果: content="In forests deep, bears roam wild and free,\nStrength and grace embodied, nature's majesty."
函数执行耗时: 1.7084482000209391 秒


## 理解 RunnablePassthrough的作用和两种用法

In [1]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [4]:
runnable = RunnableParallel(
    
    chain=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3).assign(triple =lambda x: x["mult"] * 3),
   
)

runnable.invoke({"num": 1})

{'chain': {'num': 1, 'mult': 3, 'triple': 9}}